# Python Web Scraping Note Book
--- Jun-30-2020, Alan, Shaolun.du@gmail.com

This is a detailed python web scraping tool for China's commodity future data (includes price{open, high, low, close}, volume, OPI and exchange inventory data) and then we combine stock data from Yahoo Finance, foreign exchange data from US st.louis federal reserve to generate an integrated trading panel.

## 1-Exchange website scraping
There are 4 major commodity exchange in China, Zhengzhou exchange, Shanghai exchange, Dalian exchange and Zhongjin exchange. Most of tickers traded in Zhengzhou are agricultural commodity, Shanghai is more focused on metals, Dalian is more interested in Chemical products and Zhongjin is special for financial index products.

Their corresponding websites below:
* Shanghai exchange: http://www.shfe.com.cn/.
* Zhengzhou exchange: http://www.czce.com.cn/.
* Dalian exchange: http://www.dce.com.cn/.
* Zhongjin exchange: http://www.cffex.com.cn/.

In the following, I will walk you through both price scraping and inventory data scraping directly from those exchanges above. Then we will store those data into local database for further usage. Also I will talk about how to calculate price index and we can show side-by-side plot with price index and other technical indicators.

In [1]:
""" Dependent library imports
"""
import json
import requests
import pandas as pd
import datetime,re
from abc import ABC, abstractmethod
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

## 1-a Zhengzhou exchange
### Daily exchange price data (open, high, low, close, vol, opi)
Daily exchange price data located in Zhengzhou exchange website-->daily trading information--> pick a date to display. Please also refer to this website as an example: http://www.czce.com.cn/cn/jysj/mrhq/H770301index_1.htm

![title](img/ZZ_web.jpg)
The above screenshot is their daily market trading information, we will extract the table of market in this picture using this request format:"http://www.czce.com.cn/cn/DFSStaticFiles/Future/{Year}/{Date}/FutureDataDaily.htm"

The way to find this url for requesting via inspection on current web and look into "network" tab. Whenever a new request come in, we will get a file transfered from network and you will find the corresponding file address listed above.

For example a simple requests url with date "2020-06-29" looks like below: "http://www.czce.com.cn/cn/DFSStaticFiles/Future/2020/20200629/FutureDataDaily.htm"

Now, we will use python request with beautiful soup module to parse this table.

In [2]:
""" This is date list generator for all exchanges url format
"""
def gen_proc_params(exc, start_date_str, end_date_str):
    def check_date_format(date_str):
        if re.match(r"^\d{4}-\d{2}-\d{2}$", date_str):
            return True
        else:
            return False
    date_list = []
    if check_date_format(start_date_str) and \
        check_date_format(end_date_str):
        year_start, month_start, day_start = start_date_str.split("-")
        year_end, month_end, day_end = end_date_str.split("-")
        start_date = datetime.date(int(year_start),
                                   int(month_start),
                                   int(day_start))
        end_date = datetime.date(int(year_end),
                                 int(month_end),
                                 int(day_end))
        delta_days = (end_date-start_date).days
        i = 0
        if delta_days>=0:
            while i <= delta_days:
                date = start_date+datetime.timedelta(days=i)
                if exc == "ZZ":
                    date_list.append([date.strftime('%Y'),date.strftime('%Y%m%d')])
                elif exc == "SH":
                    date_list.append(date.strftime('%Y%m%d'))
                elif exc == "DL":
                    date_list.append([int(date.strftime('%Y')),int(date.strftime('%m'))-1,int(date.strftime('%d')),date])
                elif exc == "ZJ":
                    date_list.append([date.strftime('%Y%m'),date.strftime('%d')])
                elif exc == "SH_inv":
                    date_list.append([date.strftime("%Y"),date.strftime('%Y%m%d'),date])
                else:
                    print("Exchange code:{} error".format(exc))
                    raise
                i += 1
            return date_list
        else:
            print("input params end date is earlier than start_date")
            raise
    else:
        print("Date format incorrect. Should be yyyy-mm-dd.")
        raise

We also will build a prototype of exchange parser with abstract functions outline

In [3]:
class exc_parser(ABC):
    @abstractmethod
    def _download(self, *args, **kwargs): 
        # Download market information from exchange
        pass
    @abstractmethod
    def _get_data_df(self, *args, **kwargs): 
        # Return dataframe
        pass
    @abstractmethod
    def _get_URL_TEMP(self, *args, **kwargs): 
        # Return url template
        pass
    @abstractmethod
    def _read_html_format(self, *args, **kwargs):
        # Return data frame after read html
        pass

In [4]:
class ZZ_parser(exc_parser):
    """ Zhengzhou exchange parser
    """
    def __init__( self ):
        self.__col_names = ["Dates","Code","Open","High","Low","Close","OPI","Vol"]
        self.__exc_name = "ZZ"
        self.__datas     = []
    def _get_URL_TEMP(self, cur_Y):
        # update exchange url
        if cur_Y >= 2015:
            URL_TEMPL = "http://www.czce.com.cn/cn/DFSStaticFiles/Future/{}/{}/FutureDataDaily.htm"
        elif cur_Y < 2015:
            URL_TEMPL = "http://www.czce.com.cn/cn/exchange/{}/datadaily/{}.htm"
        else:
            print("Year not found.")
            raise
        return URL_TEMPL
    def _read_html_format(self, cur_Y, page):
        if cur_Y < 2018:
            # Minor table format changes since 2018
            df = pd.read_html(page,skiprows=1,attrs={"id":"senfe"})[0]
        else:
            df = pd.read_html(page,skiprows=0)[0]
        return df
    def _download(self,sdate,edate):
        print("Exchange ZZ--->")
        # Start downloading given period
        dates_li = gen_proc_params(self.__exc_name,sdate,edate)
        ans = pd.DataFrame()
        with requests.Session() as s:
            # Open request session
            for dates in dates_li:
                print(dates)
                URL_TEMPL = self._get_URL_TEMP(int(dates[0]))
                url = URL_TEMPL.format(dates[0],dates[1])
                try:
                    page = s.get(url).text
                    df = self._read_html_format(int(dates[0]),page)
                except:
                    continue
                df.columns = [i for i in range(14)]
                df["Dates"] = dates[1]
                df = df[["Dates",0,2,3,4,5,10,9]]
                df.columns = self.__col_names
                df = df.dropna()
                # Delete some non ASCII rows scrpped
                df = df[df["Code"].str.len()<=10]
                ans = ans.append(df)
        self.__datas = ans
    def _get_data_df(self):
        # Convert output format
        self.__datas["Dates"] = pd.to_datetime(self.__datas["Dates"]).dt.date
        self.__datas = self.__datas.set_index("Dates")
        self.__datas["Code"] = self.__datas["Code"].astype(str)
        self.__datas[["Open","High","Low","Close","OPI","Vol"]] = self.__datas[["Open","High","Low","Close","OPI","Vol"]].astype(float)
        return self.__datas

In [5]:
# Testing run below
sdate = '2020-06-28'
edate = '2020-06-30'
ZZ = ZZ_parser()
ZZ._download(sdate,edate)
raw_data = ZZ._get_data_df()
print(raw_data)

Exchange ZZ--->
['2020', '20200628']
['2020', '20200629']
['2020', '20200630']
             Code    Open    High     Low   Close       OPI       Vol
Dates                                                                
2020-06-29  AP007  6170.0  6395.0  6120.0  6364.0    1529.0    1550.0
2020-06-29  AP010  7923.0  7970.0  7814.0  7932.0  108989.0  191122.0
2020-06-29  AP011  7700.0  7775.0  7640.0  7737.0    3492.0     866.0
2020-06-29  AP012  7749.0  7784.0  7668.0  7761.0    4718.0     319.0
2020-06-29  AP101  7749.0  7809.0  7682.0  7770.0   26560.0   13736.0
...           ...     ...     ...     ...     ...       ...       ...
2020-06-30  ZC102     0.0     0.0     0.0     0.0       1.0       0.0
2020-06-30  ZC103   522.8   529.0   522.8   528.4     684.0     131.0
2020-06-30  ZC104     0.0     0.0     0.0     0.0       0.0       0.0
2020-06-30  ZC105   517.0   524.8   516.8   522.6     339.0      93.0
2020-06-30  ZC106     0.0     0.0     0.0     0.0       0.0       0.0

[350 rows 

Now, let's move on to Zhengzhou exchange inventory data scraping.

To parse inventory, the data table structure is a little bit complicated than marekt price data. We will use BeautifulSoup to parse it. The parser structure looks exactly like the market price parser we wrote above, but with some special characters handling(Chinese characters).

We wil do the same thing as we did before, namely implement the above exc_parser to be an exchange inventory parser.
### Weekly exchange inventory data (Total Inventory)

In [6]:
class ZZ_inv_parser(exc_parser):
    def __init__(self):
        self.__contract_code = { "白糖SR":"SR", "一号棉CF":"CF",
                                 "菜粕RM":"RM", "菜籽油OI":"OI",
                                 "PTA":"TA",    "甲醇MA":"MA",
                                 "玻璃FG":"FG", "动力煤ZC":"ZC" }
        self.__exc_name = "ZZ"
        self.__datas = []
    def _download(self, sdate,edate ): 
        # Download market information from exchange
        print("Exchange ZZ downloading...")
        date_list = gen_proc_params(self.__exc_name, sdate, edate)
        datas = []
        for date_str in date_list:
            print(date_str)
            url_temp = self._get_URL_TEMP(int(date_str[0]))
            url = url_temp.format(date_str[0],date_str[1])
            resp = requests.get(url)
            if resp.status_code == 404:
                continue
            elif resp.status_code != 200:
                print("the resp status code of date({}) is {}".format(date_str, resp.status_code))
            page = resp.content.decode('utf-8')
            self._read_html_format(int(date_str[0]), page, datas, date_str[1])
        df = pd.DataFrame(datas)
        self.__datas = df
    def _get_URL_TEMP(self, cur_Y):
        # update exchange url
        if cur_Y > 2015:
            URL_TEMPL = "http://www.czce.com.cn/cn/DFSStaticFiles/Future/{}/{}/FutureDataWhsheet.htm"
        elif cur_Y <= 2015:
            URL_TEMPL = "http://www.czce.com.cn/cn/exchange/{}/datawhsheet/{}.htm"
        else:
            print("Year not found.")
            raise
        return URL_TEMPL
    def _read_html_format(self, cur_Y, page, datas, date_str):
        # Return data frame after read html
        soup = BeautifulSoup(page, 'html.parser')
        tables = soup.findAll("table")
        for table in tables:
            if cur_Y > 2017:
                # Format to get contract code changed since 2017
                try:
                    contract = table.b.get_text().split(u"单位")[0].strip().split(u"：")[-1]
                except:
                    continue
            elif cur_Y <= 2017 and table.b:
                contract = table.b.get_text().split(u"单位")[0].strip().split(u"：")[-1]
            idx = 0 # Column index for inventory
            for tag in table.findAll("tr")[1].findAll("td"):
                if u"仓单数量" in tag.get_text():
                    break
                else:
                    idx += 1
            if contract in self.__contract_code.keys():
                contract = self.__contract_code[contract]
                for row in table.findAll("tr"):
                    if row.findAll("td")[0].get_text() == u"总计":
                        loc_t = row.findAll("td")[idx]
                        if len(loc_t.findAll("td")) == 0:
                            invent = int(float((loc_t.get_text())))
                        else:
                            invent = int(float(loc_t.findAll("td")[0].get_text()))
                        datas.append({"Product":contract,"Dates":date_str,"INV":invent})
                        break
        return 0
    def _get_data_df(self):
        self.__datas = self.__datas.dropna()
        self.__datas["Dates"] = pd.to_datetime(self.__datas["Dates"]).dt.date
        self.__datas = self.__datas.set_index("Dates")
        self.__datas["Product"] = self.__datas["Product"].astype(str)
        return self.__datas

In [7]:
# Testing run below
sdate = '2020-06-28'
edate = '2020-06-30'
ZZ_inv = ZZ_inv_parser()
ZZ_inv._download(sdate,edate)
raw_data = ZZ_inv._get_data_df()
print(raw_data)

Exchange ZZ downloading...
['2020', '20200628']
['2020', '20200629']
['2020', '20200630']
           Product     INV
Dates                     
2020-06-29      SR   11251
2020-06-29      CF   21245
2020-06-29      RM    1870
2020-06-29      OI     300
2020-06-29      TA  221777
2020-06-29      MA    5000
2020-06-29      FG       0
2020-06-29      ZC       0
2020-06-30      SR   11201
2020-06-30      CF   21069
2020-06-30      RM    1870
2020-06-30      OI     300
2020-06-30      TA  222692
2020-06-30      MA    6682
2020-06-30      FG       0
2020-06-30      ZC       0


## 1-b Shanghai exchange
I will write in short since they all have similar structure and we are using almost the same program to scrap. 

Start from here, we also do the market information scraping first and then exchange inventory scraping next. 

For Shanghai exchange, it has a very good format and this format never changed in the past which makes us super easy to scrap and use. See this link here, take 20200623 as example: http://www.shfe.com.cn/data/dailydata/20200623.dat 
![title](img/SH_web.jpg)

In [10]:
class SH_parser(exc_parser):
    """ Shanghai exchange parser
    """
    def __init__( self ):
        self.__col_names = ["Dates","Code","Open","High","Low","Close","OPI","Vol"]
        self.__exc_name = "SH"
        self.__datas     = []
    def _get_URL_TEMP(self):
        # update exchange url
        URL_TEMPL = "http://www.shfe.com.cn/data/dailydata/{}.dat"
        return URL_TEMPL
    def _read_html_format(self,dates, jsonObj, temp_ans):
        tradingday = dates
        for idx, l in enumerate(jsonObj['o_cursor']):
            if not re.match(r'\S+\d\d\d\d', l['INSTRUMENTID']):
                continue
            try:
                temp_ans.append([ tradingday, l['INSTRUMENTID'].strip(),
                                  float(l['OPENPRICE']), float(l['HIGHESTPRICE']),
                                  float(l['LOWESTPRICE']), float(l['CLOSEPRICE']),
                                  float(l['OPENINTEREST']), float(l['VOLUME']) ])
            except:
                continue
        return temp_ans
    def _download(self,sdate,edate):
        print("Exchange SH--->")
        # Start downloading given period
        dates_li = gen_proc_params(self.__exc_name,sdate,edate)
        temp_ans = []
        with requests.Session() as s:
            # Open request session
            for dates in dates_li:
                print(dates)
                URL_TEMPL = self._get_URL_TEMP()
                url = URL_TEMPL.format(dates)
                resp = s.get(url)
                jsonObj = json.loads(resp.content.decode('utf-8'))
                temp_ans = self._read_html_format(dates, jsonObj, temp_ans)
        self.__datas = pd.DataFrame(temp_ans)
        self.__datas.columns = self.__col_names
    def _get_data_df(self):
        # Convert output format
        self.__datas = self.__datas.dropna()
        self.__datas["Dates"] = pd.to_datetime(self.__datas["Dates"]).dt.date
        self.__datas = self.__datas.set_index("Dates")
        self.__datas["Code"] = self.__datas["Code"].astype(str)
        return self.__datas

In [11]:
# Testing run below
sdate = '2020-06-28'
edate = '2020-06-30'
SH = SH_parser()
SH._download(sdate,edate)
raw_data = SH._get_data_df()
print(raw_data)

Exchange SH--->
20200628
20200629
20200630
              Code     Open     High      Low    Close       OPI       Vol
Dates                                                                     
2020-06-29  cu2007  48040.0  48720.0  48040.0  48500.0   66864.0   48989.0
2020-06-29  cu2008  48050.0  48590.0  48030.0  48380.0  120364.0  101302.0
2020-06-29  cu2009  47860.0  48470.0  47860.0  48290.0   74780.0   36893.0
2020-06-29  cu2010  47730.0  48350.0  47730.0  48150.0   50624.0   14996.0
2020-06-29  cu2011  47750.0  48210.0  47750.0  48080.0   16217.0    3351.0
...            ...      ...      ...      ...      ...       ...       ...
2020-06-30  sp2012   4604.0   4604.0   4452.0   4482.0   11936.0   13166.0
2020-06-30  sp2101   4512.0   4536.0   4506.0   4518.0    6670.0    3583.0
2020-06-30  sp2102   4544.0   4562.0   4542.0   4552.0       8.0      50.0
2020-06-30  sp2103   4592.0   4592.0   4592.0   4592.0       3.0       1.0
2020-06-30  sp2105   4662.0   4674.0   4642.0   4674.0   

### Weekly exchange inventory data (Total Inventory)
Now let's get all Shanghai exchange inventory data as well.

In [12]:
class SH_inv_parser(exc_parser):
    def __init__(self):
        self.__contract_code = { "铜":"cu","铝":"al","锌":"zn","铅":"pb",
                                 "镍":"ni","锡":"sn","天然橡胶":"ru",
                                 "沥青仓库":"bu_warehouse","沥青厂库":"bu_factory",
                                 "螺纹钢":"rb","热轧卷板":"hc",
                                 "黄金":"au","白银":"ag","原油":"sc",
                                 "线材":"zl", "中质含硫原油":"sc",
                                 "燃料油":"fu","纸浆":"sp","20号胶":"nr",
                               }
        self.__exc_name = "SH_inv"
        self.__col_names = ['Dates', 'Product', 'INV']
        self.__datas = []
    def _get_URL_TEMP(self):
        # update exchange url
        URL_TEMPL = "http://www.shfe.com.cn/data/dailydata/{}dailystock.dat"
        return URL_TEMPL
    def _download(self, sdate,edate ):
        # Download market information from exchange
        print("Exchange SH inv downloading...")
        date_list = gen_proc_params(self.__exc_name,sdate,edate)
        datas = []
        for date_str in date_list:
            print(date_str)
            URL_TEMP = self._get_URL_TEMP()
            url = URL_TEMP.format(date_str[1])
            resp = requests.get(url)
            if resp.status_code == 404:
                continue
            elif resp.status_code != 200:
                print("the resp status code of date({}) is {}".format(date_str[0:2], resp.status_code))
            jsonObj = json.loads(resp.content.decode('utf-8'))
            datas = self._read_html_format(jsonObj, date_str, datas)
        df = pd.DataFrame(datas)
        try:
            df.columns = self.__col_names
        except:
            df = pd.DataFrame(columns = self.__col_names)
        # Over write to english product name
        df = df.replace({"Product":self.__contract_code})
        self.__datas = df
    def _read_html_format(self,jsonObj, date_str,datas):
        for idx, l in enumerate(jsonObj['o_cursor']):
            # Pay attention to gold with different format
            if re.match(r'\S+?\$\$GOLD$', l['VARNAME']):
                datas.append([date_str[2], l['VARNAME'].split('$$')[0],float(l['WRTWGHTS'])])
            if not re.match(r'\S+?\$\$Total$', l['WHABBRNAME']):
                continue
            datas.append([date_str[2], l['VARNAME'].split('$$')[0],float(l['WRTWGHTS'])])
        return datas
    def _get_data_df(self):
        self.__datas = self.__datas.dropna()
        self.__datas["Dates"] = pd.to_datetime(self.__datas["Dates"]).dt.date
        self.__datas = self.__datas.set_index("Dates")
        self.__datas["Product"] = self.__datas["Product"].astype(str)
        return self.__datas

In [13]:
# Testing run below
sdate = '2020-06-28'
edate = '2020-06-30'
SH_inv = SH_inv_parser()
SH_inv._download(sdate,edate)
raw_data = SH_inv._get_data_df()
print(raw_data)

Exchange SH inv downloading...
['2020', '20200628', datetime.date(2020, 6, 28)]
['2020', '20200629', datetime.date(2020, 6, 29)]
['2020', '20200630', datetime.date(2020, 6, 30)]
                 Product         INV
Dates                               
2020-06-29            cu     36417.0
2020-06-29            al     90404.0
2020-06-29            zn     37225.0
2020-06-29            pb     16887.0
2020-06-29            ni     28191.0
2020-06-29            sn      2621.0
2020-06-29            au      1890.0
2020-06-29            ag   2098926.0
2020-06-29            rb      4741.0
2020-06-29            zl         0.0
2020-06-29            hc     12837.0
2020-06-29            sc  34670000.0
2020-06-29            fu    536380.0
2020-06-29  bu_warehouse     72000.0
2020-06-29    bu_factory    264680.0
2020-06-29            ru    229840.0
2020-06-29            nr     51076.0
2020-06-29            sp     92654.0
2020-06-30            cu     36763.0
2020-06-30            al     89827.0
2020-06-

## 1-c Dalian exchange
The next section we will finish Dalian exchange parser, it is a little complicated compared to Zhengzhou and Shanghai exchange. We will have to do post method in order to get reponse of data.

The Dalian website looks like this: http://www.dce.com.cn/dalianshangpin/xqsj/tjsj26/rtj/rxq/index.html

![title](img/SH_web.jpg)
The reponse data always has the same file name as "dayQuotesCh.html". We have to use post method to tell the server return which date's data. To do that we define a web payload structure, the object paramaters looks a little different than before.

In [14]:
class DL_parser(exc_parser):
    """ Shanghai exchange parser
    """
    def __init__( self ):
        self.__col_names = ["Dates","Code","Open","High","Low","Close","OPI","Vol"]
        self.__exc_name  = "DL"
        self.__URL_TEMPL = "http://www.dce.com.cn/publicweb/quotesdata/dayQuotesCh.html"
        self.__headers   = { 'Content-Type': 'application/x-www-form-urlencoded',
                             'Cookie': 'JSESSIONID=34581314E8E6F047ABE7D22180DCE3A2; WMONID=-b8uBX4vHDi; Hm_lvt_a50228174de2a93aee654389576b60fb=1567732473,1568333912,1568936184,1569113640; Hm_lpvt_a50228174de2a93aee654389576b60fb=1569113660',
                             'Referer': 'http://www.dce.com.cn/publicweb/quotesdata/dayQuotesCh.html',
                             'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
                           }
        self.__payload  = { 'dayQuotes.variety': 'all',
                           'dayQuotes.trade_type': '0',
                           'year': 0,
                           'month':0,
                           'day':  0,
                         }
        self.__name_map = {"豆一":"a","豆二":"b","乙二醇":"eg","焦煤":"jm","焦炭":"j",
                           "铁矿石":"i","聚氯乙烯":"pvc","聚丙烯":"pp","聚乙烯":"pe","豆粕":"m",
                           "豆油":"y","棕榈油":"p","鸡蛋":"jd","玉米淀粉":"cs","玉米":"c"}
        self.__datas     = []
    def _get_URL_TEMP(self):
        # update exchange url
        return self.__URL_TEMPL
    def _read_html_format(self,page,dates):
        df = pd.read_html(page,skiprows=0)[0]
        df.iloc[:,0] = df.iloc[:,0].map(self.__name_map)
        df = df.dropna()
        df["Dates"] = str(dates[0])+"{:02d}".format(dates[1]+1)+"{:02d}".format(dates[2])
        df["Code"]  = df.iloc[:,0]+df.iloc[:,1].astype(int).astype(str)
        df["Open"]  = df.iloc[:,2]
        df["High"]  = df.iloc[:,3]
        df["Low"]   = df.iloc[:,4]
        df["Close"] = df.iloc[:,5]
        df["OPI"]   = df.iloc[:,11]
        df["Vol"]   = df.iloc[:,10]
        df = df[["Dates","Code","Open","High","Low","Close","OPI","Vol"]]
        return df 
    def _download(self,sdate,edate):
        print("Exchange DL--->")
        # Start downloading given period
        dates_li = gen_proc_params(self.__exc_name,sdate,edate)
        ans = pd.DataFrame()
        with requests.Session() as s:
            # Open request session
            for dates in dates_li:
                print(dates)
                self.__payload['year'] = dates[0]
                self.__payload['month'] = dates[1]
                self.__payload['day'] = dates[2]
                page = s.post( self.__URL_TEMPL, data=self.__payload, headers=self.__headers).text
                try:
                    df = self._read_html_format(page,dates)
                except:
                    continue
                ans = ans.append(df)
        self.__datas = ans
    def _get_data_df(self):
        # Convert output format
        self.__datas = self.__datas.dropna()
        self.__datas["Dates"] = pd.to_datetime(self.__datas["Dates"]).dt.date
        self.__datas = self.__datas.set_index("Dates")
        self.__datas["Code"] = self.__datas["Code"].astype(str)
        return self.__datas

In [15]:
# Testing run below
sdate = '2020-06-28'
edate = '2020-06-30'
DL = DL_parser()
DL._download(sdate,edate)
raw_data = DL._get_data_df()
print(raw_data)

Exchange DL--->
[2020, 5, 28, datetime.date(2020, 6, 28)]
[2020, 5, 29, datetime.date(2020, 6, 29)]
[2020, 5, 30, datetime.date(2020, 6, 30)]
             Code  Open  High   Low   Close     OPI     Vol
Dates                                                      
2020-06-29  a2007  5418  5438  5352  5438.0     561      81
2020-06-29  a2009  4698  4858  4670  4839.0  165912  273937
2020-06-29  a2011  4320  4459  4317  4418.0     617     153
2020-06-29  a2101  4326  4435  4321  4420.0   51244   37659
2020-06-29  a2103  4447  4462  4447  4462.0      53       6
...           ...   ...   ...   ...     ...     ...     ...
2020-06-30  y2011  5674  5730  5674  5688.0     412      28
2020-06-30  y2012     -     -     -  5736.0      22       0
2020-06-30  y2101  5658  5736  5658  5704.0  189287   52462
2020-06-30  y2103     -     -     -  5874.0       9       0
2020-06-30  y2105  5728  5758  5712  5726.0   16604    3659

[308 rows x 7 columns]


### Weekly exchange inventory data (Total Inventory)
Now let's get all Dalian exchange inventory data as well.

In [16]:
class DL_inv_parser(exc_parser):
    def __init__(self):
        self.__col_names = ["Product","Location","Y-D Inv","INV","Chg"]
        self.__exc_name  = "DL"
        self.__URL_TEMPL = "http://www.dce.com.cn/publicweb/quotesdata/wbillWeeklyQuotes.html"
        self.__headers   = { 'Content-Type': 'application/x-www-form-urlencoded',
                             'Cookie': 'JSESSIONID=B2D36827C18F04E470A15A12B7C75AE5; WMONID=Zzot0IEoeuA; Hm_lvt_a50228174de2a93aee654389576b60fb=1569244935,1569337963,1569432080; Hm_lpvt_a50228174de2a93aee654389576b60fb=1569432127',
                             'Referer': 'http://www.dce.com.cn/publicweb/quotesdata/wbillWeeklyQuotes.html',
                             'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
                         }
        self.__payload  = { 'wbillWeeklyQuotes.variety': 'all',
                            'year': 0,
                            'month':0,
                            'day':  0,
                          }
        self.__name_map = {"豆一":"a","豆二":"b","乙二醇":"eg","焦煤":"jm","焦炭":"j",
                           "铁矿石":"i","聚氯乙烯":"pvc","聚丙烯":"pp","聚乙烯":"pe","豆粕":"m",
                           "豆油":"y","棕榈油":"p","鸡蛋":"jd","玉米淀粉":"cs","玉米":"c",
                          "苯乙烯":"eb","纤维板":"fb"}
        self.__datas     = []
    def _get_URL_TEMP(self):
        # update exchange url
        return self.__URL_TEMPL
    def _read_html_format(self,page,dates):
        df = pd.read_html(page,skiprows=0)[0]
        df.columns = self.__col_names
        df = df.fillna("NA")
        df = df[df["Product"].str.contains("小计")]
        df["Product"] = df["Product"].str.replace("小计", "")
        df = df.replace({"Product":self.__name_map})[["Product","INV"]]
        df["Dates"] = dates[3]
        df = df[["Dates","Product","INV"]]
        return df 
    def _download(self,sdate,edate):
        print("Exchange DL Inv--->")
        # Start downloading given period
        dates_li = gen_proc_params(self.__exc_name,sdate,edate)
        ans = pd.DataFrame()
        with requests.Session() as s:
            # Open request session
            for dates in dates_li:
                print(dates)
                self.__payload['year'] = dates[0]
                self.__payload['month'] = dates[1]
                self.__payload['day'] = dates[2]
                page = s.post( self.__URL_TEMPL, data=self.__payload, headers=self.__headers).text
                try:
                    df = self._read_html_format(page,dates)
                except:
                    continue
                ans = ans.append(df)
        self.__datas = ans
    def _get_data_df(self):
        # Convert output format
        self.__datas = self.__datas.dropna()
        self.__datas["Dates"] = pd.to_datetime(self.__datas["Dates"]).dt.date
        self.__datas = self.__datas.set_index("Dates")
        self.__datas["Product"] = self.__datas["Product"].astype(str)
        return self.__datas

In [17]:
# Testing run below
sdate = '2020-06-28'
edate = '2020-06-30'
DL_Inv = DL_inv_parser()
DL_Inv._download(sdate,edate)
raw_data = DL_Inv._get_data_df()
print(raw_data)

Exchange DL Inv--->
[2020, 5, 28, datetime.date(2020, 6, 28)]
[2020, 5, 29, datetime.date(2020, 6, 29)]
[2020, 5, 30, datetime.date(2020, 6, 30)]
           Product    INV
Dates                    
2020-06-29       a      8
2020-06-29       c  48957
2020-06-29      cs    123
2020-06-29      eb   1198
2020-06-29      eg   9364
2020-06-29      fb    201
2020-06-29      jd     44
2020-06-29      pe    252
2020-06-29       m  22358
2020-06-29       p    500
2020-06-29      pp    163
2020-06-29     pvc    461
2020-06-29       y  15655
2020-06-30       a      8
2020-06-30       b   1600
2020-06-30       c  46899
2020-06-30      cs    123
2020-06-30      eb      0
2020-06-30      eg   9364
2020-06-30      fb    201
2020-06-30      jd      0
2020-06-30      pe    431
2020-06-30       m  22358
2020-06-30       p    500
2020-06-30      pp    163
2020-06-30     pvc    461
2020-06-30       y  15655


## 1-d Zhongjin exchange
This is the last section for exchange python scraping. Zhongjin exchange is located in Shanghai and they have similar website layout. Also the responce structure is similar in JASON format.

Zhongjin exchange website looks like below: http://www.cffex.com.cn/
![title](img/ZJ_web.jpg)
The good thing for Zhongjin is they do not have any inventory. Since they only trade financial commodity futures like G-bond futures and stock index futures.

In [18]:
class ZJ_parser(exc_parser):
    """ Shanghai exchange parser
    """
    def __init__( self ):
        self.__URL_TEMPL = "http://www.cffex.com.cn/sj/hqsj/rtj/{}/{}/index.xml"
        self.__headers   = ["Dates","Code","Open","High","Low","Close","OPI","Vol"]
        self.__exc_name  = "ZJ"
        self.__datas     = []
    def _get_URL_TEMP(self):
        # update exchange url
        return self.__URL_TEMPL
    def _read_html_format(self,page, df):
        etree = ET.fromstring(page)
        for i in etree.iter(tag='dailydata'):
            df = df.append(
                pd.Series([ i.find('tradingday').text,i.find('instrumentid').text,
                            i.find('openprice').text,i.find('highestprice').text,
                            i.find('lowestprice').text,i.find('closeprice').text,
                            i.find('openinterest').text,i.find('volume').text], index=self.__headers),
                            ignore_index=True)
        return df 
    def _download(self,sdate,edate):
        print("Exchange ZJ--->")
        # Start downloading given period
        import pandas as pd
        import xml.etree.ElementTree as ET
        import requests
        df = pd.DataFrame(columns=self.__headers)
        dates_li = gen_proc_params(self.__exc_name,sdate,edate)
        with requests.Session() as s:
            for dates in dates_li:
                print(dates)
                url = self.__URL_TEMPL.format(dates[0],dates[1])
                page = s.get(url).text
                try:
                    df = self._read_html_format(page, df)
                except:
                    continue
        self.__datas = df
    def _get_data_df(self):
        # Convert output format
        self.__datas = self.__datas.dropna()
        self.__datas["Dates"] = pd.to_datetime(self.__datas["Dates"]).dt.date
        self.__datas = self.__datas.set_index("Dates")
        self.__datas["Code"] = self.__datas["Code"].astype(str)
        return self.__datas

In [19]:
# Testing run below
sdate = '2020-06-28'
edate = '2020-06-30'
ZJ = ZJ_parser()
ZJ._download(sdate,edate)
raw_data = ZJ._get_data_df()
print(raw_data)

Exchange ZJ--->
['202006', '28']
['202006', '29']
['202006', '30']
              Code     Open     High      Low    Close    OPI    Vol
Dates                                                               
2020-06-29  IC2007   5729.4     5738   5668.8   5710.6  89387  70963
2020-06-29  IC2008   5657.4   5671.8   5603.2   5641.2   2225   1245
2020-06-29  IC2009   5600.2   5607.8     5536   5572.8  59774  13823
2020-06-29  IC2012     5426   5435.6     5370   5407.2  34823   6163
2020-06-29  IF2007   4100.8   4108.6   4052.2     4078  90974  73040
...            ...      ...      ...      ...      ...    ...    ...
2020-06-30  TF2012  101.165  101.285  101.085  101.285   1844    252
2020-06-30  TF2103  100.645   100.66   100.55   100.66    273     24
2020-06-30  TS2009    101.1   101.23    101.1   101.22  15857   5852
2020-06-30  TS2012  100.985   101.04   100.93  101.035   3368    302
2020-06-30  TS2103  100.725  100.725  100.725  100.725     33      2

[496 rows x 7 columns]
